#Data_Agg
this notebook gets the 
team 1 seed: team 1 plus minius : team 2 seed : team 2 plus minus : who won

This file creates the average plus minus for each teamID over the chosen period of data
and it also creates a df of the team 1 team 2 matchup/who won and what their average seeding was.

In [1]:
%matplotlib inline 
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import matplotlib as plt
import copy

In [2]:
team_data = pd.read_csv('MTeams.csv')
seed_data = pd.read_csv('MNCAATourneySeeds.csv', index_col = "TeamID")
reg_season_data = pd.read_csv('MRegularSeasonCompactResults.csv')


In [3]:
team_data.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021
1,1102,Air Force,1985,2021
2,1103,Akron,1985,2021
3,1104,Alabama,1985,2021
4,1105,Alabama A&M,2000,2021


In [4]:

seed_data.head(100)


,Season,Seed
TeamID,,
1207,1985,W01
1210,1985,W02
1228,1985,W03
1260,1985,W04
1374,1985,W05
...,...,...
1299,1986,X16
1246,1986,Y01
1210,1986,Y02


In [5]:
reg_season_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [6]:
team_nameID = team_data['TeamID']
team_and_ID = pd.DataFrame(data=team_nameID)
team_and_ID['TeamName'] = team_data['TeamName']

team_scores = reg_season_data['WTeamID']
Score_ID = pd.DataFrame(data=team_scores)
Score_ID['WScore'] = reg_season_data['WScore']
Score_ID['LTeamID'] = reg_season_data['LTeamID']
Score_ID['LScore'] = reg_season_data['LScore']

Score_ID['plus_minus_W'] = Score_ID['WScore'] - Score_ID['LScore']
Score_ID['plus_minus_L'] = Score_ID['LScore'] - Score_ID['WScore']

W_ID = Score_ID['WTeamID']
df_W_plusminus = pd.DataFrame(data=W_ID)
df_W_plusminus['plus_minus_W'] = Score_ID['plus_minus_W']
df_W_plusminus.rename(index = {'index': 'TeamID'}, columns = {'WTeamID': 'TeamID', 'plus_minus_W': 'plus_minus'}, inplace= True)

L_ID = Score_ID['LTeamID']
df_L_plusminus = pd.DataFrame(data=L_ID)
df_L_plusminus['plus_minus_L'] = Score_ID['plus_minus_L']
df_L_plusminus.rename(index = {'index': 'TeamID'}, columns = {'LTeamID': 'TeamID', 'plus_minus_L': 'plus_minus'}, inplace= True)

Score_ID_grouped_W = Score_ID.groupby('WTeamID')['plus_minus_W'].mean()
Score_ID_grouped_L = Score_ID.groupby('LTeamID')['plus_minus_L'].mean()
Score_ID_grouped_df_W = pd.DataFrame(data=Score_ID_grouped_W)
Score_ID_grouped_df_L = pd.DataFrame(data=Score_ID_grouped_L)

plusminus_df = pd.concat([df_W_plusminus,df_L_plusminus], ignore_index=True)

plusminus_df2 = plusminus_df.groupby('TeamID')['plus_minus'].sum()

plusminus_team = pd.merge(plusminus_df2, team_and_ID, on = "TeamID")

plusminus_team = plusminus_team.set_index('TeamID')

plusminus_team.head(20)



,plus_minus,TeamName
TeamID,,
1101,-673,Abilene Chr
1102,-3339,Air Force
1103,2755,Akron
1104,4812,Alabama
1105,-3315,Alabama A&M
1106,-3320,Alabama St
1107,-104,SUNY Albany
1108,-7009,Alcorn St
1109,-2261,Alliant Intl


In [7]:
Score_ID.head(10)

,WTeamID,WScore,LTeamID,LScore,plus_minus_W,plus_minus_L
0,1228,81,1328,64,17,-17
1,1106,77,1354,70,7,-7
2,1112,63,1223,56,7,-7
3,1165,70,1432,54,16,-16
4,1192,86,1447,74,12,-12
5,1218,79,1337,78,1,-1
6,1228,64,1226,44,20,-20
7,1242,58,1268,56,2,-2
8,1260,98,1133,80,18,-18
9,1305,97,1424,89,8,-8


In [8]:
import random

who_won_df = Score_ID['WTeamID']
who_won_df = pd.DataFrame(data=who_won_df)
who_won_df['LTeamID'] = Score_ID['LTeamID']        

who_won_df['Who_won'] = 1

who_won_df.rename(columns = {'WTeamID': 'Team_1_ID', "LTeamID":'Team_2_ID'}, inplace= True)
            
#random number generator
team1 = []
team2 = []
winner = []
for i in range(len(who_won_df)):
    number = random.randint(1,101)
    if number%2 == 0:
        team1.append(who_won_df['Team_1_ID'].iloc[i])
        team2.append(who_won_df['Team_2_ID'].iloc[i])
        winner.append(1)
    elif number%2 != 0:
        team1.append(who_won_df['Team_2_ID'].iloc[i])
        team2.append(who_won_df['Team_1_ID'].iloc[i])
        winner.append(-1)
        
who_won_df_random = pd.DataFrame(team1)
who_won_df_random['Team_2_ID'] = team2
who_won_df_random['Winner'] = winner
who_won_df_random.rename(columns = {0:'Team_1_ID'}, inplace=True)
    
team_IDs = list(plusminus_team.index)

avg_plusminus_1 = []
for i in who_won_df_random['Team_1_ID']:
    if i in team_IDs:
        avg_plusminus_1.append(plusminus_team['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')
        
avg_plusminus_2 = []
for i in who_won_df_random['Team_2_ID']:
    if i in team_IDs:
        avg_plusminus_2.append(plusminus_team['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')

who_won_df_random['Team1_plusminus'] = avg_plusminus_1
who_won_df_random['Team2_plusminus'] = avg_plusminus_2
who_won_df_random

# team_IDs_seeds = list(seed_data.index)

# seed_1 = []
# for i in who_won_df['Team_1_ID']:
#     if i in team_IDs_seeds:
#         seed_1.append(seed_data['Seed'].loc[i])
#     else: 
#         seed_1.append('NA')

# who_won_df['Seed_Team_1'] = seed_1

# seed_2 = []
# for i in who_won_df['Team_2_ID']:
#     if i in team_IDs_seeds:
#         seed_2.append(seed_data['Seed'].loc[i])
#     else: 
#         seed_2.append('NA')

# who_won_df['Seed_Team_1'] = seed_1
# who_won_df['Seed_Team_2'] = seed_2




,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus
0,1228,1328,1,7109,8588
1,1106,1354,1,-3320,-2997
2,1112,1223,1,11519,-3369
3,1432,1165,-1,-288,-3027
4,1447,1192,-1,-1625,-2407
...,...,...,...,...,...
166875,1204,1209,1,-809,-622
166876,1402,1111,1,-2366,-1950
166877,1299,1224,1,-3730,-6941
166878,1393,1314,1,9176,11851


In [9]:
def data_agg(team_data, reg_season_data, seed_data, year):
    # import team data to get teams 
#     print(team_data)
    ### create data frame for the teams and their ID - for later 
    team_nameID = team_data['TeamID']
    team_and_ID = pd.DataFrame(data=team_nameID)
    team_and_ID['TeamName'] = team_data['TeamName']
#     print(team_and_ID)
    
    ### create a data frame of the data for scores and teams and then find plus minus
#     print(reg_season_data)
    team_scores = reg_season_data['WTeamID']
    Score_ID = pd.DataFrame(data=team_scores)
    Score_ID['WScore'] = reg_season_data['WScore']
    Score_ID['LTeamID'] = reg_season_data['LTeamID']
    Score_ID['LScore'] = reg_season_data['LScore']
#     print(Score_ID)

    Score_ID['plus_minus_W'] = Score_ID['WScore'] - Score_ID['LScore']
    Score_ID['plus_minus_L'] = Score_ID['LScore'] - Score_ID['WScore']
    
    W_ID = Score_ID['WTeamID']
    df_W_plusminus = pd.DataFrame(data=W_ID)
    df_W_plusminus['plus_minus_W'] = Score_ID['plus_minus_W']
    df_W_plusminus.rename(index = {'index': 'TeamID'}, columns = {'WTeamID': 'TeamID', 'plus_minus_W': 'plus_minus'}, inplace= True)

    L_ID = Score_ID['LTeamID']
    df_L_plusminus = pd.DataFrame(data=L_ID)
    df_L_plusminus['plus_minus_L'] = Score_ID['plus_minus_L']
    df_L_plusminus.rename(index = {'index': 'TeamID'}, columns = {'LTeamID': 'TeamID', 'plus_minus_L': 'plus_minus'}, inplace= True)
    
    #create data frame for team IDs and their corresponding plus minus score
    Score_ID_grouped_W = Score_ID.groupby('WTeamID')['plus_minus_W'].mean()
    Score_ID_grouped_L = Score_ID.groupby('LTeamID')['plus_minus_L'].mean()
    Score_ID_grouped_df_W = pd.DataFrame(data=Score_ID_grouped_W)
    Score_ID_grouped_df_L = pd.DataFrame(data=Score_ID_grouped_L)

    plusminus_df = pd.concat([df_W_plusminus,df_L_plusminus], ignore_index=True)
    
    #get sum of the plus minus over the entire season for each team
    plusminus_df2 = plusminus_df.groupby('TeamID')['plus_minus'].sum()
    
    #merge team name and ID with plus minus score
    plusminus_team = pd.merge(plusminus_df2, team_and_ID, on = "TeamID")

    plusminus_team = plusminus_team.set_index('TeamID')
#     print(plusminus_team)
    
    import random
    
    #create dataframe of winning and losing team and whether they won = end up not using
    who_won_df = Score_ID['WTeamID']
    who_won_df = pd.DataFrame(data=who_won_df)
    who_won_df['LTeamID'] = Score_ID['LTeamID']        

    who_won_df['Who_won'] = 1

    who_won_df.rename(columns = {'WTeamID': 'Team_1_ID', "LTeamID":'Team_2_ID'}, inplace= True)
    
    ### use a random number generator to split up team 1 and team 2 and put corresponding value of who won
    team1 = []
    team2 = []
    winner = []
    for i in range(len(who_won_df)):
        number = random.randint(1,101)
        if number%2 == 0:
            team1.append(who_won_df['Team_1_ID'].iloc[i])
            team2.append(who_won_df['Team_2_ID'].iloc[i])
            winner.append(1)
        elif number%2 != 0:
            team1.append(who_won_df['Team_2_ID'].iloc[i])
            team2.append(who_won_df['Team_1_ID'].iloc[i])
            winner.append(-1)
    
    #turn above lists into dataframes
    who_won_df_random = pd.DataFrame(team1)
    who_won_df_random['Team_2_ID'] = team2
    who_won_df_random['Winner'] = winner
    who_won_df_random.rename(columns = {0:'Team_1_ID'}, inplace=True)

    team_IDs = list(plusminus_team.index)
    
    #find plus minus value for each team based on their team ID
    avg_plusminus_1 = []
    for i in who_won_df_random['Team_1_ID']:
        if i in team_IDs:
            avg_plusminus_1.append(plusminus_team['plus_minus'].loc[i])
        else: 
            avg_plusminus_1.append('NA')

    avg_plusminus_2 = []
    for i in who_won_df_random['Team_2_ID']:
        if i in team_IDs:
            avg_plusminus_2.append(plusminus_team['plus_minus'].loc[i])
        else: 
            avg_plusminus_2.append('NA')
    
#     Team_names = []
#     for i in who_won_df_random['Team_1_ID']:
#         if i in team_IDs:
#             avg_plusminus_1.append(['plus_minus'].loc[i])
#         else: 
#             avg_plusminus_1.append('NA')
    
    #add it all into the data frame
    who_won_df_random['Team1_plusminus'] = avg_plusminus_1
    who_won_df_random['Team2_plusminus'] = avg_plusminus_2
    who_won_df_random

    #### 2020 doesnt have seed data so we took out the seed aspect
    # team_IDs_seeds = list(seed_data.index)

    # seed_1 = []
    # for i in who_won_df['Team_1_ID']:
    #     if i in team_IDs_seeds:
    #         seed_1.append(seed_data['Seed'].loc[i])
    #     else: 
    #         seed_1.append('NA')

    # who_won_df['Seed_Team_1'] = seed_1

    # seed_2 = []
    # for i in who_won_df['Team_2_ID']:
    #     if i in team_IDs_seeds:
    #         seed_2.append(seed_data['Seed'].loc[i])
    #     else: 
    #         seed_2.append('NA')

    # who_won_df['Seed_Team_1'] = seed_1
    # who_won_df['Seed_Team_2'] = seed_2


    ### print what year the data is from
    who_won_df_random['Year'] = year

    #return the data frame for the data given
    return who_won_df_random


In [10]:
reg_season_data2019 = reg_season_data[156090:161553]
seed_data2019 = seed_data[2219:]
team_data = pd.read_csv('MTeams.csv')

In [11]:
df_2019 = data_agg(team_data, reg_season_data2019, seed_data2019, 2019)
df_2019

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1168,1113,-1,-61,152,2019
1,1265,1119,-1,-61,-88,2019
2,1120,1375,1,350,-56,2019
3,1232,1123,-1,-97,70,2019
4,1454,1130,-1,-192,-60,2019
...,...,...,...,...,...,...
5458,1209,1426,1,88,-43,2019
5459,1277,1276,1,452,402,2019
5460,1382,1387,-1,105,116,2019
5461,1217,1463,-1,34,201,2019


In [12]:
reg_season_data2018 = reg_season_data[150685:156090]
seed_data2018 = seed_data[2154:2219]
team_data = pd.read_csv('MTeams.csv')

In [13]:
df_2018 = data_agg(team_data, reg_season_data2018, seed_data2018, 2018)
df_2018

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1233,1107,-1,122,168,2018
1,1319,1112,-1,-409,330,2018
2,1226,1113,-1,-58,254,2018
3,1116,1359,1,189,-232,2018
4,1120,1313,1,322,-84,2018
...,...,...,...,...,...,...
5400,1348,1172,-1,265,284,2018
5401,1426,1209,-1,120,213,2018
5402,1397,1246,-1,258,219,2018
5403,1335,1217,1,189,51,2018


In [14]:
reg_season_data2017 = reg_season_data[145290:150685]
seed_data2017 = seed_data[2083:2154]
team_data = pd.read_csv('MTeams.csv')

In [15]:
df_2017 = data_agg(team_data, reg_season_data2017, seed_data2017, 2017)
df_2017

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1107,1336,1,158,-33,2017
1,1277,1112,-1,111,368,2017
2,1113,1340,1,-102,-12,2017
3,1236,1116,-1,115,197,2017
4,1316,1120,-1,-184,27,2017
...,...,...,...,...,...,...
5390,1463,1343,-1,67,246,2017
5391,1433,1348,-1,281,280,2017
5392,1153,1374,-1,464,500,2017
5393,1407,1402,1,123,56,2017


In [16]:
total_df = pd.concat([df_2019,df_2018,df_2017], ignore_index=True)
total_df

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1168,1113,-1,-61,152,2019
1,1265,1119,-1,-61,-88,2019
2,1120,1375,1,350,-56,2019
3,1232,1123,-1,-97,70,2019
4,1454,1130,-1,-192,-60,2019
...,...,...,...,...,...,...
16258,1463,1343,-1,67,246,2017
16259,1433,1348,-1,281,280,2017
16260,1153,1374,-1,464,500,2017
16261,1407,1402,1,123,56,2017


In [17]:
team_data2 = pd.read_csv('MTeams.csv')
seed_data2 = pd.read_csv('MNCAATourneySeeds_with_2021csv.csv', index_col = "TeamID")
reg_season_data2 = pd.read_csv('MRegularSeasonCompactResults_2021.csv')

In [18]:
seed_data2021 = seed_data2[2287:]
reg_season_data2021 = reg_season_data2[166882:]

In [19]:
df_2021 = data_agg(team_data, reg_season_data2021, seed_data2021, 2021)
df_2021

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1203,1298,-1,-71,81,2021
1,1462,1324,1,97,-196,2021
2,1441,1423,1,-135,-6,2021
3,1249,1222,-1,-183,468,2021
4,1240,1104,-1,76,294,2021
...,...,...,...,...,...,...
3851,1382,1433,1,203,170,2021
3852,1159,1259,1,266,20,2021
3853,1261,1104,-1,183,294,2021
3854,1153,1222,-1,-74,468,2021


In [20]:
seed_data2020 = []
reg_season_data2020 = reg_season_data2[161553:166882]

In [21]:
df_2020 = data_agg(team_data, reg_season_data2020, seed_data2020, 2020)
df_2020

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1105,1398,-1,-301,-39,2020
1,1108,1177,-1,-130,-41,2020
2,1373,1110,1,136,64,2020
3,1276,1111,1,210,-25,2020
4,1283,1114,-1,92,100,2020
...,...,...,...,...,...,...
5324,1224,1299,-1,-431,-56,2020
5325,1314,1393,-1,-20,162,2020
5326,1233,1389,-1,-25,67,2020
5327,1190,1101,-1,270,131,2020


In [22]:
total_df = pd.concat([df_2021,df_2020,df_2019,df_2018,df_2017], ignore_index=True)
total_df

,Team_1_ID,Team_2_ID,Winner,Team1_plusminus,Team2_plusminus,Year
0,1203,1298,-1,-71,81,2021
1,1462,1324,1,97,-196,2021
2,1441,1423,1,-135,-6,2021
3,1249,1222,-1,-183,468,2021
4,1240,1104,-1,76,294,2021
...,...,...,...,...,...,...
25443,1463,1343,-1,67,246,2017
25444,1433,1348,-1,281,280,2017
25445,1153,1374,-1,464,500,2017
25446,1407,1402,1,123,56,2017


In [23]:
#2021 plusmius with team name and id
def data_agg_plusminus(team_data, reg_season_data):

    team_nameID = team_data['TeamID']
    team_and_ID = pd.DataFrame(data=team_nameID)
    team_and_ID['TeamName'] = team_data['TeamName']
#     print(team_and_ID)
    
    ### create a data frame of the data for scores and teams and then find plus minus
#     print(reg_season_data)
    team_scores = reg_season_data['WTeamID']
    Score_ID = pd.DataFrame(data=team_scores)
    Score_ID['WScore'] = reg_season_data['WScore']
    Score_ID['LTeamID'] = reg_season_data['LTeamID']
    Score_ID['LScore'] = reg_season_data['LScore']
#     print(Score_ID)

    Score_ID['plus_minus_W'] = Score_ID['WScore'] - Score_ID['LScore']
    Score_ID['plus_minus_L'] = Score_ID['LScore'] - Score_ID['WScore']
    
    W_ID = Score_ID['WTeamID']
    df_W_plusminus = pd.DataFrame(data=W_ID)
    df_W_plusminus['plus_minus_W'] = Score_ID['plus_minus_W']
    df_W_plusminus.rename(index = {'index': 'TeamID'}, columns = {'WTeamID': 'TeamID', 'plus_minus_W': 'plus_minus'}, inplace= True)

    L_ID = Score_ID['LTeamID']
    df_L_plusminus = pd.DataFrame(data=L_ID)
    df_L_plusminus['plus_minus_L'] = Score_ID['plus_minus_L']
    df_L_plusminus.rename(index = {'index': 'TeamID'}, columns = {'LTeamID': 'TeamID', 'plus_minus_L': 'plus_minus'}, inplace= True)
    
    #create data frame for team IDs and their corresponding plus minus score
    Score_ID_grouped_W = Score_ID.groupby('WTeamID')['plus_minus_W'].mean()
    Score_ID_grouped_L = Score_ID.groupby('LTeamID')['plus_minus_L'].mean()
    Score_ID_grouped_df_W = pd.DataFrame(data=Score_ID_grouped_W)
    Score_ID_grouped_df_L = pd.DataFrame(data=Score_ID_grouped_L)

    plusminus_df = pd.concat([df_W_plusminus,df_L_plusminus], ignore_index=True)
    
    #get sum of the plus minus over the entire season for each team
    plusminus_df2 = plusminus_df.groupby('TeamID')['plus_minus'].sum()
    
    #merge team name and ID with plus minus score
    plusminus_team = pd.merge(plusminus_df2, team_and_ID, on = "TeamID")

    plusminus_team = plusminus_team.set_index('TeamID')
    
    return plusminus_team

In [24]:
#plusminus dataframe for only 2021 data

df_2021_stats = data_agg_plusminus(team_data, reg_season_data2021)
df_2021_stats

,plus_minus,TeamName
TeamID,,
1101,312,Abilene Chr
1102,-330,Air Force
1103,80,Akron
1104,294,Alabama
1105,-58,Alabama A&M
...,...,...
1467,-7,Merrimack
1468,77,Bellarmine
1469,-201,Dixie St


In [25]:
X = total_df[['Team1_plusminus', 'Team2_plusminus']]
Y = total_df['Winner']


#Split data
x_train = X[0:20358]
x_train = np.asarray(x_train)
x_test =  X[20358:]
x_test = np.asarray(x_test)

print(x_train.shape)
print(x_test.shape)


y_train = Y[0:20358]
y_train = np.array(y_train)
y_train = y_train.reshape(-1)
y_test =  Y[20358:]
y_test = np.array(y_test)
y_test = y_test.reshape(-1)
print(y_train.shape)
print(y_test.shape)

(20358, 2)
(5090, 2)
(20358,)
(5090,)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

#Run a model
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
print("F1 Score - Logistic Regression:", f1_score(y_test, logreg_pred, average = 'weighted'))

F1 Score - Logistic Regression: 0.7307957320023878


# Read in and Clean up Tourney Data

In [27]:
tourney_seeds = pd.read_csv("tourney_seeds.csv")
tourney_seeds = tourney_seeds[["seed_line", "Seed", "overall_seed", "TeamID", "TeamName"]]

round_1 = pd.read_csv("Round_1_2021.csv")
round_1 = round_1[["tm1_seed_num", "tm1_Seed", "tm1_ID", "tm2_Seed", "tm2_seed_num", "tm2_ID"]]
round_1 = round_1[0:36][:]
round_1['tm2_ID'] = round_1['tm2_ID'].fillna(0)


round_1['tm1_ID'] = pd.to_numeric(round_1['tm1_ID'], downcast = 'integer')
round_1['tm2_ID'] = pd.to_numeric(round_1['tm2_ID'], downcast = 'integer')

print(tourney_seeds)
print(round_1)

    seed_line Seed  overall_seed  TeamID      TeamName
0           1  W01             4    1276      Michigan
1           2  W02             5    1104       Alabama
2           3  W03            11    1400         Texas
3           4  W04            13    1199    Florida St
4           5  W05            20    1160      Colorado
..        ...  ...           ...     ...           ...
63         12  Z12            49    1457      Winthrop
64         13  Z13            52    1317   North Texas
65         14  Z14            57    1159       Colgate
66         15  Z15            61    1331  Oral Roberts
67         16  Z16            64    1216      Hartford

[68 rows x 5 columns]
    tm1_seed_num tm1_Seed  tm1_ID tm2_Seed  tm2_seed_num  tm2_ID
0             16     W16b    1411     W16a            16    1291
1             11     X11a    1179     X11b            11    1455
2             16     X16a    1111     X16b            16    1313
3             11     W11b    1417     W11a            11 

In [28]:
#def bracketology():

#PLAY IN ROUND-------------------------------------------------------------
#carve out play in games 
round_0_sked = round_1[0:4][:]

#reformat round 1
round_1_sked = round_1[4:][:]
round_1_sked = round_1_sked.reset_index()
round_1_sked = round_1_sked.drop(['index'], axis = 1)


#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
round_0_x = round_0_sked[['tm1_ID', 'tm2_ID']]
#round_0_x

#ONCE YOU HAVE THE XDATA, run through the model
round_0_ws = logreg.predict(round_0_x)

#Attach predicted winners to the schedule
round_0_sked['pred_winner'] = round_0_ws

#Eyeball it
#print(round_0_sked)

#Pull the winners out
round_0_winners = []

for i in range(0, 4):
    if round_0_sked['pred_winner'][i] == 1:
        round_0_winners.append(round_0_sked['tm1_ID'][i])
    else:
        round_0_winners.append(round_0_sked['tm2_ID'][i])

#check the winners
#print(round_0_winners)

#print the winners names
print("First Four Winners:\n")
for i in round_0_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")

#print(round_1_sked)

#ROUND 1 -------------------------------------------------------------------

#populate the round 1 bracket with the play-in winners
round_1_sked['tm2_ID'][20] = round_0_winners[0]
round_1_sked['tm2_ID'][23] = round_0_winners[1]
round_1_sked['tm2_ID'][28] = round_0_winners[2]
round_1_sked['tm2_ID'][29] = round_0_winners[3]

round_1_sked = round_1_sked.sort_values(['tm1_Seed'])
# print(round_1_sked)

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_round_1_sked =round_1_sked['tm1_ID']
team_1_round_1_sked = pd.DataFrame(data=team_1_round_1_sked)
team_1_round_1_sked = team_1_round_1_sked.set_index('tm1_ID')
# print(team_1_round_1_sked)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_round_1_sked.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')

team_2_plusminus = []
team_2_round_1_sked = round_1_sked['tm2_ID']
team_2_round_1_sked = pd.DataFrame(data=team_2_round_1_sked)
team_2_round_1_sked = team_2_round_1_sked.set_index('tm2_ID')
# print(team_2_round_2)

plus_minus_2 = []
for i in team_2_round_1_sked.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
round_1_sked['tm1_plusminus'] = plus_minus_1
round_1_sked['tm2_plusminus'] = plus_minus_2


###########################################ENDING#########################################################################


round_1_x = round_1_sked[['tm1_plusminus', 'tm2_plusminus']]
#print(round_1_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
round_1_ws = logreg.predict(round_1_x)

#Attach predicted winners to the schedule
round_1_sked['pred_winner'] = round_1_ws
round_1_sked = round_1_sked.reset_index()
round_1_sked = round_1_sked.drop(['index'], axis = 1)

#Eyeball it
#print(round_1_sked)

#Pull the winners out    
round_1_winners = []

for i in range(0, len(round_1_sked['pred_winner'])):
    if round_1_sked['pred_winner'][i] == 1:
        round_1_winners.append(round_1_sked['tm1_ID'][i])
    else:
        round_1_winners.append(round_1_sked['tm2_ID'][i])

#check the winners
#print(round_1_winners)

#print the winners names
print("Round 1 Winners:\n")
for i in round_1_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")


#ROUND of 32 -------------------------------------------------------------------

#build the bracket
#append game numbers to winners
round2games = np.asarray([1,4,3,2,2,3,4,1])
round2games2 = round2games + 4
round2games3 = round2games2 + 4
round2games4 = round2games3 + 4

round2game_num = np.concatenate((round2games, round2games2, round2games3, round2games4))
#print(round2game_num)

round_2 = {"TeamID": round_1_winners, "game_num":round2game_num}
round_2 = pd.DataFrame(round_2)
round_2 = round_2.sort_values(['game_num'])

tms = ["tm1","tm2"]
tms = tms * 16
round_2["tms"] = tms
round_2 = round_2.pivot(index = "game_num", columns = "tms", values = "TeamID")
round_2 = round_2.reset_index()
#round_2 = round_2.drop(['index'], axis = 1)

################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_round_2 =round_2['tm1']
team_1_round_2 = pd.DataFrame(data=team_1_round_2)
team_1_round_2 = team_1_round_2.set_index('tm1')
# print(team_1_round_2)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_round_2.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')

team_2_plusminus = []
team_2_round_2 = round_2['tm2']
team_2_round_2 = pd.DataFrame(data=team_2_round_2)
team_2_round_2 = team_2_round_2.set_index('tm2')
# print(team_2_round_2)

plus_minus_2 = []
for i in team_2_round_2.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
round_2['tm1_plusminus'] = plus_minus_1
round_2['tm2_plusminus'] = plus_minus_2


###########################################ENDING#########################################################################

# print(round_2)

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
round_2_x = round_2[['tm1_plusminus', 'tm2_plusminus']]
#print(round_2_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
round_2_ws = logreg.predict(round_2_x)

#Attach predicted winners to the schedule
round_2['pred_winner'] = round_2_ws

#Eyeball it
#print(round_2)

#Pull the winners out
round_2_winners = []

for i in range(0, len(round_2['pred_winner'])):
    if round_2['pred_winner'][i] == 1:
        round_2_winners.append(round_2['tm1'][i])
    else:
        round_2_winners.append(round_2['tm2'][i])

#check the winners
#print(round_2_winners)

#print the winners names
print("Round of 32 Winners:\n")
for i in round_2_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")

#SWEET SIXTEEN -------------------------------------------------------------------

#build the bracket
sweet16 = {"TeamID": round_2_winners}
sweet16 = pd.DataFrame(sweet16)
sweet16 = pd.merge(sweet16, tourney_seeds, on ='TeamID', how ='left')
sweet16 = sweet16.drop(['seed_line', 'overall_seed', 'TeamName'], axis = 1)
#print(sweet16)

#append game numbers to winners
sweet16game_num = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8]
#print(sweet16game_num)

sweet16["game_num"] = sweet16game_num
#print(sweet16)

tms = ["tm1","tm2"]
tms = tms * 8
sweet16["tms"] = tms
#print(sweet16)
sweet16 = sweet16.pivot(index = "game_num", columns = "tms", values = "TeamID")
sweet16 = sweet16.reset_index()
#print(sweet16)


################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_sweet_16 = sweet16['tm1']
team_1_sweet_16 = pd.DataFrame(data=team_1_sweet_16)
team_1_sweet_16 = team_1_sweet_16.set_index('tm1')
# print(team_1_sweet_16)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_sweet_16.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')
        
team_2_plusminus = []
team_2_sweet_16 = sweet16['tm2']
team_2_sweet_16 = pd.DataFrame(data=team_2_sweet_16)
team_2_sweet_16 = team_2_sweet_16.set_index('tm2')
# print(team_2_sweet_16)

plus_minus_2 = []
for i in team_2_sweet_16.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
sweet16['tm1_plusminus'] = plus_minus_1
sweet16['tm2_plusminus'] = plus_minus_2

# print(sweet16)
###########################################ENDING#########################################################################

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
sweet16_x = sweet16[['tm1_plusminus', 'tm2_plusminus']]
#print(sweet16_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
sweet16_ws = logreg.predict(sweet16_x)

#Attach predicted winners to the schedule
sweet16['pred_winner'] = sweet16_ws

#Eyeball it
#print(sweet16)

#Pull the winners out
sweet16_winners = []

for i in range(0, len(sweet16['pred_winner'])):
    if sweet16['pred_winner'][i] == 1:
        sweet16_winners.append(sweet16['tm1'][i])
    else:
        sweet16_winners.append(sweet16['tm2'][i])

#check the winners
#print(sweet16_winners)

#print the winners names
print("Sweet 16 Winners:\n")
for i in sweet16_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")


#ELITE EIGHT -------------------------------------------------------------------

#build the bracket
elite8 = {"TeamID": sweet16_winners}
elite8 = pd.DataFrame(elite8)
elite8 = pd.merge(elite8, tourney_seeds, on ='TeamID', how ='left')
elite8 = elite8.drop(['seed_line', 'overall_seed', 'TeamName'], axis = 1)
#print(elite8)

#append game numbers to winners
elite8games_num = np.asarray([1,1,2,2,3,3,4,4])
elite8["game_num"] = elite8games_num
#print(elite8)

tms = ["tm1","tm2"]
tms = tms * 4
elite8["tms"] = tms
elite8 = elite8.pivot(index = "game_num", columns = "tms", values = "TeamID")
elite8 = elite8.reset_index()

################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_elite8 = elite8['tm1']
team_1_elite8 = pd.DataFrame(data=team_1_elite8)
team_1_elite8 = team_1_elite8.set_index('tm1')
# print(team_1_elite8)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_elite8.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')

team_2_plusminus = []
team_2_elite8 = elite8['tm2']
team_2_elite8 = pd.DataFrame(data=team_2_elite8)
team_2_elite8 = team_2_elite8.set_index('tm2')
# print(team_2_sweet_16)

plus_minus_2 = []
for i in team_2_elite8.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
elite8['tm1_plusminus'] = plus_minus_1
elite8['tm2_plusminus'] = plus_minus_2
###########################################ENDING#########################################################################


# print(elite8)

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
elite8_x = elite8[['tm1_plusminus', 'tm2_plusminus']]
#print(elite8_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
elite8_ws = logreg.predict(elite8_x)

#Attach predicted winners to the schedule
elite8['pred_winner'] = elite8_ws

#Eyeball it
#print(elite8)

#Pull the winners out
elite8_winners = []

for i in range(0, len(elite8['pred_winner'])):
    if elite8['pred_winner'][i] == 1:
        elite8_winners.append(elite8['tm1'][i])
    else:
        elite8_winners.append(elite8['tm2'][i])

#check the winners
#print(elite8_winners)

#print the winners names
print("Elite 8 Winners:\n")
for i in elite8_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")


#FINAL FOUR -------------------------------------------------------------------

#build the bracket
final4 = {"TeamID": elite8_winners}
final4 = pd.DataFrame(final4)
final4 = pd.merge(final4, tourney_seeds, on ='TeamID', how ='left')
final4 = final4.drop(['seed_line', 'overall_seed', 'TeamName'], axis = 1)
#print(final4)

#append game numbers to winners
final4_num = np.asarray([1,1,2,2])
final4["game_num"] = final4_num
#print(final4)

tms = ["tm1","tm2"]
tms = tms * 2
final4["tms"] = tms
final4 = final4.pivot(index = "game_num", columns = "tms", values = "TeamID")
final4 = final4.reset_index()

################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_final4 = final4['tm1']
team_1_final4 = pd.DataFrame(data=team_1_final4)
team_1_final4 = team_1_final4.set_index('tm1')
# print(team_1_final4)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_final4.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')

team_2_plusminus = []
team_2_final4 = final4['tm2']
team_2_final4 = pd.DataFrame(data=team_2_final4)
team_2_final4 = team_2_final4.set_index('tm2')
# print(team_2_sweet_16)

plus_minus_2 = []
for i in team_2_final4.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
final4['tm1_plusminus'] = plus_minus_1
final4['tm2_plusminus'] = plus_minus_2
###########################################ENDING#########################################################################

# print(final4)

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
final4_x = final4[['tm1_plusminus', 'tm2_plusminus']]
#print(final4_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
final4_ws = logreg.predict(final4_x)

#Attach predicted winners to the schedule
final4['pred_winner'] = final4_ws

#Eyeball it
#print(final4)

#Pull the winners out
final4_winners = []

for i in range(0, len(final4['pred_winner'])):
    if final4['pred_winner'][i] == 1:
        final4_winners.append(final4['tm1'][i])
    else:
        final4_winners.append(final4['tm2'][i])

#check the winners
#print(final4_winners)

#print the winners names
print("Final 4 Winners:\n")
for i in final4_winners:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")

#CHAMPIONSHIP ROUND -------------------------------------------------------------------

#build the bracket
champ = {"TeamID": final4_winners}
champ = pd.DataFrame(champ)
champ = pd.merge(champ, tourney_seeds, on ='TeamID', how ='left')
champ = champ.drop(['seed_line', 'overall_seed', 'TeamName'], axis = 1)
#print(champ)

#append game numbers to winners
champ_num = np.asarray([1,1])
champ["game_num"] = champ_num
#print(champ)

tms = ["tm1","tm2"]
champ["tms"] = tms
champ = champ.pivot(index = "game_num", columns = "tms", values = "TeamID")
champ = champ.reset_index()

################################################ Mikayla Beginning ########################################################
#adding in plusminus into each teamID in round dataframe
team_1_plusminus = []
team_1_champ = champ['tm1']
team_1_champ = pd.DataFrame(data=team_1_champ)
team_1_champ = team_1_champ.set_index('tm1')
# print(team_1_champ)

team_IDs = list(plusminus_team.index)
    
#find plus minus value for each team based on their team ID
plus_minus_1 = []
for i in team_1_champ.index:
    if i in team_IDs:
        plus_minus_1.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_1.append('NA')

team_2_plusminus = []
team_2_champ = champ['tm2']
team_2_champ = pd.DataFrame(data=team_2_champ)
team_2_champ = team_2_champ.set_index('tm2')
# print(team_2_champ)

plus_minus_2 = []
for i in team_2_champ.index:
    if i in team_IDs:
        plus_minus_2.append(df_2021_stats['plus_minus'].loc[i])
    else: 
        avg_plusminus_2.append('NA')
        
champ['tm1_plusminus'] = plus_minus_1
champ['tm2_plusminus'] = plus_minus_2
###########################################ENDING#########################################################################

# print(champ)

#NEED THIS TO INLCUDE CONNECTING THE PLUST MINUSES AND THEN RUNNING THOSE THROUGH THE MODEL
champ_x = champ[['tm1_plusminus', 'tm2_plusminus']]
#print(champ_x)
                         
#ONCE YOU HAVE THE XDATA, run through the model
champ_ws = logreg.predict(champ_x)

#Attach predicted winners to the schedule
champ['pred_winner'] = champ_ws

# #Eyeball it
#print(champ)

#Pull the winners out
champ_winner = []

for i in range(0, len(champ['pred_winner'])):
    if champ['pred_winner'][i] == 1:
        champ_winner.append(champ['tm1'][i])
    else:
        champ_winner.append(champ['tm2'][i])

#check the winners
#print(champ_winner)

#print the winners names
print("National Champion:\n")
for i in champ_winner:
    print(tourney_seeds.loc[tourney_seeds["TeamID"] == i, 'TeamName'].iloc[0])
print("\n\n")

First Four Winners:

TX Southern
Wichita St
Norfolk St
UCLA



Round 1 Winners:

Michigan
Alabama
Abilene Chr
Florida St
Colorado
BYU
Connecticut
St Bonaventure
Gonzaga
Iowa
E Washington
Virginia
UC Santa Barbara
USC
Oregon
Oklahoma
Illinois
Houston
West Virginia
Liberty
Tennessee
San Diego St
Clemson
Loyola-Chicago
Baylor
Ohio St
Arkansas
North Texas
Winthrop
Utah St
Virginia Tech
North Carolina



Round of 32 Winners:

Michigan
Colorado
Abilene Chr
Alabama
Gonzaga
UC Santa Barbara
USC
Iowa
Loyola-Chicago
Liberty
San Diego St
Houston
Baylor
Winthrop
Arkansas
Ohio St



Sweet 16 Winners:

Colorado
Abilene Chr
Gonzaga
Iowa
Loyola-Chicago
Houston
Baylor
Arkansas



Elite 8 Winners:

Abilene Chr
Gonzaga
Houston
Baylor



Final 4 Winners:

Gonzaga
Houston



National Champion:

Gonzaga





<ipython-input-28-e4cff5d2bd7e>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  round_1_sked['tm2_ID'][20] = round_0_winners[0]
<ipython-input-28-e4cff5d2bd7e>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  round_1_sked['tm2_ID'][23] = round_0_winners[1]
<ipython-input-28-e4cff5d2bd7e>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  round_1_sked['tm2_ID'][28] = round_0_winners[2]
<ipython-input-28-e4cff5d2bd7e>:52: SettingWithCo

In [29]:
round_1_sked.sort_values(['tm1_Seed'])

,tm1_seed_num,tm1_Seed,tm1_ID,tm2_Seed,tm2_seed_num,tm2_ID,tm1_plusminus,tm2_plusminus,pred_winner
0,1,W01,1276,W16,16,1411,261,80,1
1,2,W02,1104,W15,15,1233,294,107,1
2,3,W03,1400,W14,14,1101,168,312,-1
3,4,W04,1199,W13,13,1422,190,179,1
4,5,W05,1160,W12,12,1207,293,17,1
5,6,W06,1140,W11,11,1417,218,113,1
6,7,W07,1163,W10,10,1268,174,69,1
7,8,W08,1261,W09,9,1382,183,203,-1
8,1,X01,1211,X16,16,1313,598,96,1
9,2,X02,1234,X15,15,1213,343,192,1


In [30]:
round2games = np.asarray([1,4,3,2,2,3,4,1])
round2games2 = round2games + 4
round2games3 = round2games2 + 4
round2games4 = round2games3 + 4

round2game_num = np.concatenate((round2games, round2games2, round2games3, round2games4))
round2game_num

array([ 1,  4,  3,  2,  2,  3,  4,  1,  5,  8,  7,  6,  6,  7,  8,  5,  9,
       12, 11, 10, 10, 11, 12,  9, 13, 16, 15, 14, 14, 15, 16, 13])

In [31]:
tourney_seeds

,seed_line,Seed,overall_seed,TeamID,TeamName
0,1,W01,4,1276,Michigan
1,2,W02,5,1104,Alabama
2,3,W03,11,1400,Texas
3,4,W04,13,1199,Florida St
4,5,W05,20,1160,Colorado
...,...,...,...,...,...
63,12,Z12,49,1457,Winthrop
64,13,Z13,52,1317,North Texas
65,14,Z14,57,1159,Colgate
66,15,Z15,61,1331,Oral Roberts


In [32]:
tourney_seeds1 = pd.merge(tourney_seeds, df_2021_stats, on ='TeamName', how ='left') 
print(tourney_seeds)
print(tourney_seeds1)

    seed_line Seed  overall_seed  TeamID      TeamName
0           1  W01             4    1276      Michigan
1           2  W02             5    1104       Alabama
2           3  W03            11    1400         Texas
3           4  W04            13    1199    Florida St
4           5  W05            20    1160      Colorado
..        ...  ...           ...     ...           ...
63         12  Z12            49    1457      Winthrop
64         13  Z13            52    1317   North Texas
65         14  Z14            57    1159       Colgate
66         15  Z15            61    1331  Oral Roberts
67         16  Z16            64    1216      Hartford

[68 rows x 5 columns]
    seed_line Seed  overall_seed  TeamID      TeamName  plus_minus
0           1  W01             4    1276      Michigan         261
1           2  W02             5    1104       Alabama         294
2           3  W03            11    1400         Texas         168
3           4  W04            13    1199    Flori

In [33]:
# Create dataframe of possible 2021 tournament matchups
tourney_ids = list(copy.deepcopy(tourney_seeds["TeamID"].sort_values()))

team1_list = []
for i in tourney_ids:
    for j in range(68):
        team1_list.append(i)

team2_list = []
for x in range(68):
    for y in tourney_ids:
        team2_list.append(y)
        
tourney_matchups1 = pd.DataFrame(data={"Team1":team1_list, "Team2":team2_list})
tourney_matchups2 = copy.deepcopy(tourney_matchups1[tourney_matchups1["Team1"] < tourney_matchups1["Team2"]])
#print(tourney_matchups2.head(5))
#print(len(tourney_matchups2))


#append model inputs, then figure out the difference value by doing team one minus team to....



#run through the model and get predict proba
#Kaggle only wants team ids and predicted probability of a team 1 win, team ids are 2021_TEAM1ID_TEAM2ID


#submit file with two columns and predicted probability in column two.....column titles "ID" "PRED" ----2278 rows


In [34]:
print(tourney_matchups2)

      Team1  Team2
1      1101   1104
2      1101   1111
3      1101   1116
4      1101   1124
5      1101   1140
...     ...    ...
4418   1452   1457
4419   1452   1458
4486   1455   1457
4487   1455   1458
4555   1457   1458

[2278 rows x 2 columns]


In [35]:
match_w_dat = pd.merge(tourney_matchups2, tourney_seeds1, left_on='Team1', right_on='TeamID', how = "left")
match_w_dat = match_w_dat.drop(['seed_line', 'Seed', 'overall_seed', 'TeamID', 'TeamName'], axis = 1)
#match_w_dat = match_w_dat.rename({"plus_minus":"Team1_pm"})
match_w_dat2 = pd.merge(tourney_matchups2, tourney_seeds1, left_on='Team2', right_on='TeamID', how = "left")
match_w_dat2 = match_w_dat2.drop(['seed_line', 'Seed', 'overall_seed', 'TeamID', 'TeamName'], axis = 1)
print(match_w_dat)

print(match_w_dat2)
match_w_dat['tm2_pm'] = match_w_dat2['plus_minus']
print(match_w_dat)


      Team1  Team2  plus_minus
0      1101   1104         312
1      1101   1111         312
2      1101   1116         312
3      1101   1124         312
4      1101   1140         312
...     ...    ...         ...
2273   1452   1457         140
2274   1452   1458         140
2275   1455   1457          50
2276   1455   1458          50
2277   1457   1458         306

[2278 rows x 3 columns]
      Team1  Team2  plus_minus
0      1101   1104         294
1      1101   1111          -2
2      1101   1116         328
3      1101   1124         431
4      1101   1140         218
...     ...    ...         ...
2273   1452   1457         306
2274   1452   1458         154
2275   1455   1457         306
2276   1455   1458         154
2277   1457   1458         154

[2278 rows x 3 columns]
      Team1  Team2  plus_minus  tm2_pm
0      1101   1104         312     294
1      1101   1111         312      -2
2      1101   1116         312     328
3      1101   1124         312     431
4      1101

In [36]:
sub_x_dat = match_w_dat[['plus_minus',"tm2_pm"]]
sub_x_dat

sub_ws = logreg.predict(sub_x_dat)
sub_proba = logreg.predict_proba(sub_x_dat)
print(sub_ws)
print(sub_proba)
print(sub_proba[0][1])
print(sub_proba[1][1])
print(sub_proba.shape)
probas = []
for i in range(0, 2278):
    probas.append(sub_proba[i][1])
    
#print(probas)

strings = []

match_w_dat["ID"] = "2021_" + match_w_dat["Team1"].astype(str) + "_" + match_w_dat["Team2"].astype(str)
print(match_w_dat)

match_w_dat["PRED"] = probas

print(match_w_dat)

submission = match_w_dat[["ID", "PRED"]]
print(submission)


submission.to_csv("SPN_bracket_predictions_v1.csv")

[ 1  1 -1 ... -1 -1  1]
[[0.48698611 0.51301389]
 [0.15519047 0.84480953]
 [0.53409343 0.46590657]
 ...
 [0.81192984 0.18807016]
 [0.65003863 0.34996137]
 [0.31094361 0.68905639]]
0.513013893605307
0.8448095339416418
(2278, 2)
      Team1  Team2  plus_minus  tm2_pm              ID
0      1101   1104         312     294  2021_1101_1104
1      1101   1111         312      -2  2021_1101_1111
2      1101   1116         312     328  2021_1101_1116
3      1101   1124         312     431  2021_1101_1124
4      1101   1140         312     218  2021_1101_1140
...     ...    ...         ...     ...             ...
2273   1452   1457         140     306  2021_1452_1457
2274   1452   1458         140     154  2021_1452_1458
2275   1455   1457          50     306  2021_1455_1457
2276   1455   1458          50     154  2021_1455_1458
2277   1457   1458         306     154  2021_1457_1458

[2278 rows x 5 columns]
      Team1  Team2  plus_minus  tm2_pm              ID      PRED
0      1101   1104     

In [37]:
# Kaggle scoring function
def logloss(prob_pred, results):
    score1 = 0
    for i in range(len(prob_pred)):
        game_score = results[i]*np.log(prob_pred[i]) + (1-results[i])*np.log(1-prob_pred[i])
        score1 += game_score
    score2 = score1 * -1 / len(prob_pred)
    return score2

results1a = pd.read_csv("ncaa_tourney_2021_results1a.csv")
results1b = results1a.merge(submission, left_on="ID", right_on="ID")
results1c = copy.deepcopy(results1b[["ID","PRED","Result"]])
print(results1a)
print(results1c)

prob_pred_test2 = list(results1c["PRED"])
results_test2 = list(results1c["Result"])
print(" ")
print("Kaggle score is: ")
print(logloss(prob_pred_test2, results_test2))

                ID  Result
0   2021_1211_1313       1
1   2021_1281_1328       0
2   2021_1166_1364       1
3   2021_1325_1438       1
4   2021_1179_1425       0
..             ...     ...
58  2021_1116_1124       0
59  2021_1222_1333       1
60  2021_1211_1417       1
61  2021_1124_1222       1
62  2021_1124_1211       1

[63 rows x 2 columns]
                ID      PRED  Result
0   2021_1211_1313  0.938856       1
1   2021_1281_1328  0.361440       0
2   2021_1166_1364  0.481493       1
3   2021_1325_1438  0.433800       1
4   2021_1179_1425  0.518611       0
..             ...       ...     ...
58  2021_1116_1124  0.349859       0
59  2021_1222_1333  0.928758       1
60  2021_1211_1417  0.933211       1
61  2021_1124_1222  0.436427       1
62  2021_1124_1211  0.273488       1

[63 rows x 3 columns]
 
Kaggle score is: 
0.6389409599565341


In [98]:
#Find Kaggle Score for 2019

tourneys = pd.read_csv("MNCAATourneySeeds.csv")
tourney_2019 = tourneys[tourneys["Season"] == 2019]
#print(tourney_2019)


df_2019_stats = data_agg_plusminus(team_data, reg_season_data2019)
#print(df_2019_stats)

tourney_ids_2019 = list(copy.deepcopy(tourney_2019["TeamID"].sort_values()))
#print(tourney_ids_2019)

team1_list = []
for i in tourney_ids_2019:
    for j in range(68):
        team1_list.append(i)

team2_list = []
for x in range(68):
    for y in tourney_ids_2019:
        team2_list.append(y)

tourney_matchups1 = pd.DataFrame(data={"Team1":team1_list, "Team2":team2_list})
tourney_matchups2 = copy.deepcopy(tourney_matchups1[tourney_matchups1["Team1"] < tourney_matchups1["Team2"]])

#print(tourney_matchups2)

tourney_seeds1 = pd.merge(tourney_2019, df_2019_stats, on ='TeamID', how ='left') 
#print(tourney_seeds)
#print(tourney_seeds1)

match_w_dat = pd.merge(tourney_matchups2, tourney_seeds1, left_on='Team1', right_on='TeamID', how = "left")
match_w_dat = match_w_dat.drop(['Seed', 'TeamID', 'TeamName'], axis = 1)
#match_w_dat = match_w_dat.rename({"plus_minus":"Team1_pm"})
match_w_dat2 = pd.merge(tourney_matchups2, tourney_seeds1, left_on='Team2', right_on='TeamID', how = "left")
match_w_dat2 = match_w_dat2.drop(['Seed', 'TeamID', 'TeamName'], axis = 1)
#print(match_w_dat)

#print(match_w_dat2)
match_w_dat['tm2_pm'] = match_w_dat2['plus_minus']
#print(match_w_dat)

sub_x_dat = match_w_dat[['plus_minus',"tm2_pm"]]
sub_x_dat

sub_ws = logreg.predict(sub_x_dat)
sub_proba = logreg.predict_proba(sub_x_dat)
#print(sub_ws)
#print(sub_proba)
#print(sub_proba[0][1])
#print(sub_proba[1][1])
#print(sub_proba.shape)
probas = []
for i in range(0, 2278):
    probas.append(sub_proba[i][1])
    
#print(probas)

strings = []

match_w_dat["ID"] = "2019_" + match_w_dat["Team1"].astype(str) + "_" + match_w_dat["Team2"].astype(str)
#print(match_w_dat)

match_w_dat["PRED"] = probas

#print(match_w_dat)

submission = match_w_dat[["ID", "PRED"]]
#print(submission)


# Kaggle scoring function
def logloss(prob_pred, results):
    score1 = 0
    for i in range(len(prob_pred)):
        game_score = results[i]*np.log(prob_pred[i]) + (1-results[i])*np.log(1-prob_pred[i])
        score1 += game_score
    score2 = score1 * -1 / len(prob_pred)
    return score2

############CREATE 2019 RESULTS FILE ----------------------
results = pd.read_csv("MNCAATourneyCompactResults.csv")
results_2019 = results[results['Season'] == 2019]

#print(results_2019)


matchups1 = "2019_" + results_2019["WTeamID"].astype(str) + "_" + results_2019["LTeamID"].astype(str)
matchups2 = "2019_" + results_2019["LTeamID"].astype(str) + "_" + results_2019["WTeamID"].astype(str)

matches = pd.DataFrame()
matches["ID"] = matchups1[:]
print(matches.shape)

res1 = []
for i in range(0,67):
    res1.append(1)
#print(len(res1))

matches["Result"] = res1

res2 = []
for i in range(0,67):
    res2.append(0)
#print(len(res1))

matches2 = pd.DataFrame()
matches2["ID"] = matchups2[:]
matches2["Result"] = res2

#print(matches)
#print(matches2)

results_2019 = matches.append(matches2)
print(results_2019)

######_____________________________________

results1a = results_2019
results1b = submission.merge(results_2019, left_on="ID", right_on="ID")
results1c = copy.deepcopy(results1b[["ID","PRED","Result"]])
print(results1a)
print(results1c)

prob_pred_test2 = list(results1c["PRED"])
results_test2 = list(results1c["Result"])
print(" ")
print("Kaggle score is: ")
print(logloss(prob_pred_test2, results_test2))

(67, 1)
                  ID  Result
2184  2019_1125_1396       1
2185  2019_1192_1341       1
2186  2019_1113_1385       1
2187  2019_1295_1300       1
2188  2019_1120_1308       1
...              ...     ...
2246  2019_1246_1120       0
2247  2019_1181_1277       0
2248  2019_1277_1403       0
2249  2019_1120_1438       0
2250  2019_1403_1438       0

[134 rows x 2 columns]
                  ID  Result
2184  2019_1125_1396       1
2185  2019_1192_1341       1
2186  2019_1113_1385       1
2187  2019_1295_1300       1
2188  2019_1120_1308       1
...              ...     ...
2246  2019_1246_1120       0
2247  2019_1181_1277       0
2248  2019_1277_1403       0
2249  2019_1120_1438       0
2250  2019_1403_1438       0

[134 rows x 2 columns]
                ID      PRED  Result
0   2019_1101_1246  0.265738       0
1   2019_1113_1138  0.150443       0
2   2019_1113_1385  0.574359       1
3   2019_1120_1242  0.709846       1
4   2019_1120_1246  0.456052       1
..             ...       .

In [73]:
############CREATE 2019 RESULTS FILE ----------------------
results = pd.read_csv("MNCAATourneyCompactResults.csv")
results_2019 = results[results['Season'] == 2019]
#print(results_2019)

matchups1 = "2019_" + tourney_matchups2["Team1"].astype(str) + "_" + tourney_matchups2["Team2"].astype(str)
matchups2 = "2019_" + tourney_matchups2["Team2"].astype(str) + "_" + tourney_matchups2["Team1"].astype(str)

matches = pd.DataFrame()
matches["matchups"] = matchups1[:]
#print(matches.shape)

res1 = []
for i in range(0,2278):
    res1.append(1)
#print(len(res1))

matches["Results"] = res1

res2 = []
for i in range(0,2278):
    res2.append(0)
#print(len(res1))

matches2 = pd.DataFrame()
matches2["matchups"] = matchups2[:]
matches2["Results"] = res2

#print(matches)
#print(matches2)

results_2019 = matches.append(matches2)
#print(results_2019)

######_____________________________________

            matchups  Results
1     2019_1101_1113        1
2     2019_1101_1120        1
3     2019_1101_1124        1
4     2019_1101_1125        1
5     2019_1101_1133        1
...              ...      ...
4418  2019_1449_1459        1
4419  2019_1449_1463        1
4486  2019_1458_1459        1
4487  2019_1458_1463        1
4555  2019_1459_1463        1

[2278 rows x 2 columns]
            matchups  Results
1     2019_1113_1101        0
2     2019_1120_1101        0
3     2019_1124_1101        0
4     2019_1125_1101        0
5     2019_1133_1101        0
...              ...      ...
4418  2019_1459_1449        0
4419  2019_1463_1449        0
4486  2019_1459_1458        0
4487  2019_1463_1458        0
4555  2019_1463_1459        0

[2278 rows x 2 columns]
            matchups  Results
1     2019_1101_1113        1
2     2019_1101_1120        1
3     2019_1101_1124        1
4     2019_1101_1125        1
5     2019_1101_1133        1
...              ...      ...
4418  2019_1459_1449

### results = pd.read_csv("MNCAATourneyCompactResults.csv")
print(results)
